# Introducción

La compañía de seguros **Sure Tomorrow** quiere resolver varias tareas con la ayuda de machine learning y te pide que evalúes esa posibilidad.

## Objetivos

- **Tarea 1**: encontrar clientes que sean similares a un cliente determinado. Esto ayudará a los agentes de la compañía con el marketing.
- **Tarea 2**: predecir si es probable que un nuevo cliente reciba un beneficio de seguro. ¿Puede un modelo de predicción funcionar mejor que un modelo ficticio?
- **Tarea 3**: predecir la cantidad de beneficios de seguro que probablemente recibirá un nuevo cliente utilizando un modelo de regresión lineal.
- **Tarea 4**: proteger los datos personales de los clientes sin romper el modelo de la tarea anterior.

# Preprocesamiento de información

Importaremos la librerías necesarias a lo largo de este documento y evaluaremos la calidad de los datos proporcionados por la compañia `Sure Tomorrow`.

In [1]:
# Importando librerias
import pandas as pd
import numpy as np
from scipy.spatial.distance import euclidean, cityblock
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
# Importando datos
df = pd.read_csv('datasets/insurance_us.csv')
df.head(10)

,Gender,Age,Salary,Family members,Insurance benefits
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


In [3]:
# Información detallada de "df"
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Gender              5000 non-null   int64  
 1   Age                 5000 non-null   float64
 2   Salary              5000 non-null   float64
 3   Family members      5000 non-null   int64  
 4   Insurance benefits  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Lo único que podemos mejorar de esta información es el tipo de dato para las columnas `Age` y `Salary` pues son enteros y el nombre de las columnas pues si bien no son erroneos, por buenas prácticas estas deben ser completamente en minúsuculas, ademas de que en vez de contener espacios deben ser guiones bajos `_`. Por tal motivo procederemos con la corrección de dicha información.

In [4]:
# Renomabrando columnas de "df" y pasando los datos a valores "int"
df.rename(columns={'Gender':'gender',
                   'Age':'age',
                   'Salary':'salary',
                   'Family members':'family_members',
                   'Insurance benefits':'insurance_benefits'},inplace=True)
df = df.astype('int64')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   gender              5000 non-null   int64
 1   age                 5000 non-null   int64
 2   salary              5000 non-null   int64
 3   family_members      5000 non-null   int64
 4   insurance_benefits  5000 non-null   int64
dtypes: int64(5)
memory usage: 195.4 KB


Una vez contamos con la información adecuada en el formato correcto comenzaremos con los objetivos que solicita `Sure Tomorrow`.
# Tarea "1": Encontrar clientes que sean similares a un cliente determinado. 

De acuerdo con lo que se solicita en la **Tarea 1** debemos trabajar con distancias, estas pueden ser `euclideana` o `manhattan`, por lo que la función contendra ambas métricas.

Para poder demostrar este objetivo, crearemos la función `nearest_neighbor_predict` con la intención de que pueda determinar a que grupo puede pertenecer pero primero, debemos separar la variable objetivo del resto de datos, es decir, debemos aislar la columna `insurance_benefits` del resto.

In [5]:
# Construyendo la función "nearest_neighbor_predict"
def nearest_neighbor_predict(clients,client,quantity):
    
    X = clients.drop(columns=['insurance_benefits']).values
    y = np.array(client)
    new_df = clients.copy()
    
    values_1 = []
    values_2 = []
    for idx in range(X.shape[0]):
        values_1.append(euclidean(X[idx],y))
        values_2.append(cityblock(X[idx],y))
    
    new_df['euclidean'] = pd.Series(values_1)
    new_df['manhattan'] = pd.Series(values_2)
    
    result_1 = new_df.sort_values(by='euclidean',ascending=True)
    result_2 = new_df.sort_values(by='manhattan',ascending=True)
    
    return result_1.iloc[:quantity,[0,1,2,3]], result_2.iloc[:quantity,[0,1,2,3]]


Esta función nos permite identificar aquellos clientes similares a al que estamos tomando como referencia, ademas que nos muestra la cantidad de clientes similares que se le indique. La función `nearest_neighbor_predict` permite hacer esta identificación a través de los siguientes parámetros:

- `clients`: Es el `df` que contiene todos los clientes proporcionados por `Sure Tomorrow`. Tipo de dato `DataFrame`.
- `client`: Es el registro que tomaremos como referencia para saber cuantos de los contenidos en `df` son similares. Tipo de dato `list`
- `quantity`: La cantidad de clientes similares que nos interesan. Tipo de dato `int`.

Ahora probaremos la función con el parámetro `client = [1,43,41000,2]` y buscaremos los `7` clientes más parecidos.

In [6]:
# Aplicando "nearest_neighbor_predict"
a, b = nearest_neighbor_predict(df,[1,43,41000,2],7)
a, b

(      gender  age  salary  family_members
 5          1   43   41000               2
 1995       0   45   41000               0
 3801       0   46   41000               2
 3972       1   40   41000               1
 4763       1   46   41000               1
 2759       0   47   41000               3
 2717       0   37   41000               0,
       gender  age  salary  family_members
 5          1   43   41000               2
 3972       1   40   41000               1
 3801       0   46   41000               2
 4763       1   46   41000               1
 1995       0   45   41000               0
 2759       0   47   41000               3
 3434       1   36   41000               2)

Como podemos observar en el resultado, tenemos vectores diferentes para cada una de las métricas de distancia calculadas que son similares al vector `client`, sin embargo también podemos observar que esto puede deberse a que hay una escala muy diferente entre los valores que toman las columnas resaltando la columna `salary` con el resto siendo la que tiene valores mucho más elevados que las demas por lo que ahora abordaremos el ejercicio aplicando un metodo de escalamiento haciendo uso de la función `Standard Scaler`.

In [7]:
# Construyendo la función "nearest_neighbor_predict" aplicando escalamiento a "df"
def nearest_neighbor_predict(clients,client,quantity):
    
    scaler = StandardScaler()

    clients = clients.drop(columns=['insurance_benefits']).values

    scaler.fit(clients)
    X = scaler.transform(clients)
    client = scaler.transform(np.array(client).reshape(1,-1))
    y = client[0]
    new_df = df.copy()
    
    values_1 = []
    values_2 = []
    for idx in range(X.shape[0]):
        values_1.append(euclidean(X[idx],y))
        values_2.append(cityblock(X[idx],y))
    
    new_df['euclidean'] = pd.Series(values_1)
    new_df['manhattan'] = pd.Series(values_2)
    
    result_1 = new_df.sort_values(by='euclidean',ascending=True)
    result_2 = new_df.sort_values(by='manhattan',ascending=True)
    
    return result_1.iloc[:quantity,[0,1,2,3]], result_2.iloc[:quantity,[0,1,2,3]]

a, b = nearest_neighbor_predict(df,[1,43,41000,2],7)
a, b

(      gender  age  salary  family_members
 5          1   43   41000               2
 1147       1   42   40800               2
 4074       1   43   39600               2
 1019       1   42   39600               2
 2962       1   41   41100               2
 2128       1   45   40500               2
 106        1   45   41600               2,
       gender  age  salary  family_members
 5          1   43   41000               2
 1147       1   42   40800               2
 4074       1   43   39600               2
 2962       1   41   41100               2
 1019       1   42   39600               2
 2128       1   45   40500               2
 106        1   45   41600               2)

Aplicando el cálculo de distancia `euclidiana` y `manhattan` al escalamiento de los datos podemos observar que los registros son considerablemente más similares variando un poco en la columna `salary` mientras que en el resto son practicaménte iguales por lo que podemos concluir que hacer uso de escalamiento debe ser considerado si tenemos una diferencia considerable entre los datos que recibe cada columna.

# Tarea 2: Predecir si es probable que un nuevo cliente reciba un beneficio de seguro. 

¿Puede un modelo de predicción funcionar mejor que un modelo ficticio?. Un modelo de predicción intentaría predecir el mejor resultado mientras que un modelo ficticio es también conocido como el modelo `dummy` que es un código simple que no significa nada.

Comenzaremos explicando de que se trata la `regresión lineal` que es un modelo matemático con el objetivo de poder encontrar la línea recta que mejor se ajuste a los datos que le estamos proporcionando, es decir, buscara un ajuste que le permita tener la pendiente y posición más adecuadas en el plano que minimice totalmente el error que pueda proporcionar al predecir algún parámetro. Esta función está basada en la siguiente fórmula:

$$
a = (X \cdot w) + w_0
$$

Donde:

- X: Contiene la matriz de datos correspondiente de la información proporcionada en `df`.
- w: Cotiene un vector de pesos que permitirá que la la recta ajuste su pendiente.
- w0: Cotiene el ajuste de altura que dicha recta tendra a lo largo de sus ejes.

Comenzaremos con la creación del modelo `dummy` y evaluaremos su rendimiento.

In [8]:
# Creando la clase "dummy_regression"
class dummy_regression():
    def fit(self,train_features,train_target):
        self.train_features = train_features
        self.train_target = train_target
    def predict(self,test_features):
        return np.zeros(test_features.shape[0])

In [9]:
# Verificando funcionamiento de "dummy_regression"
features = df.drop(columns=['insurance_benefits'],axis=1)
target = df['insurance_benefits']

model = dummy_regression()
model.fit(features,target)
predictions = model.predict(features)
print('r2_score:',r2_score(target,predictions))

r2_score: -0.1021184544233924


Como podemos ver, un modelo ficticio nos arroja un resultado negativo en la evaluación, esto nos indica que el ajuste de la recta que es capaz de modelar basado en los datos que le proporcionamos no esta ni cerca de ser el mejor ajuste para predecir de forma eficiente. Ahora crearemos la función `linear_regressión` para verificar si este modelo logra tener un mejor rendimiento que el ficticio.

Basados en la formula presentada al inicio de esta sección y al trabajar con vectores en vez de valores escalares, debemos trabajar dichos vectores mejor conocidos como `matrices`. Nuestra matriz de datos es aquella que contiene toda la información referente a nuestras características, es decir lo que guardamos en la variable `features` que analogo a la fórmula es `X`. Por otro lado tenemos a `w` que es un vector de pesos, estos pesos no son tán fáciles de encontrar pues deben ser de acuerdo a la información proporcionada por `features` que matematicamente estaría dado por la siguiente fórmula:

$$
w = (X^T X)^{-1} X^T y
$$

Por lo que debemos trabajar con matrices transpuestas e inversas. Afortunadamente todas estos procesos podemos encontrarlos en python por lo que construiremos la clase y procederemos a evaluarla de la misma manera.

In [10]:
class linear_regression():
    def fit(self,train_features,train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = (np.linalg.inv(X.T.dot(X)).dot(X.T)).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

In [11]:
# Verificando funcionamiento de "dummy_regression"
model = linear_regression()
model.fit(features,target)
predictions = model.predict(features)
print('r2_score:',r2_score(target,predictions))

r2_score: 0.42494550308169177


Como podemos observar si bien no tenemos el mejor ajuste, este modelo tiene la capacidad de predecir con mejor precisión que el modelo ficticio por lo que contestanto a la pregunta:

- **¿Puede un modelo de predicción funcionar mejor que un modelo ficticio?**

Esto es lógico puesto que de forma matemática cumplimos con todos los parámetros de la formula de `regresión lineal` que se acompla una una `regresión líneal múltiple` que es aquella que trabaja con matrices y no valores escalares lo que nos permite poder considerar más de una característica de nustros datos y poder modelar una línea en un hiperplano.

# Tarea 3: Predecir la cantidad de beneficios de seguro que probablemente recibirá un nuevo cliente utilizando un modelo de regresión lineal.

Ahora probaremos los modelos anteriores contra los modelos de regresión lineal que nos propociona `scikit learn` con la intención de evaluar nuevamente el rendimiento que estos nos proporcionan en su forma más básica, es decir, con los hiperparámetros ajustados a los valores por default.

## Regresión lineal

Comenzaremos con el más básico que es la `LinearRegression`.

In [12]:
# Entrenando modelo
lr_model = LinearRegression()
lr_model.fit(features,target)

LinearRegression()

Prodeceremos con la evaluación de `lr_model` para la cuál aplicaremos la validación cruzada para esto con la intención de obtener un mejor parámetro de desempeño al cambiar los bloques de validación.

In [13]:
# Evaluando con validación cruzada
score = cross_val_score(lr_model,features,target,cv=5)
print('Desempeño:',np.mean(score))

Desempeño: 0.42311376920775334


De acuerdo con los valores que arroja de desempeño tanto la `regresión lineal` de `sklearn` como la implementada en la clase `linear_regression` son muy similares, sin embargo, `linear_regression` es un poco más precisa pues tiene un `score` ligeramente más elevado.

## Árboles de decisión

Continuaremos ahora con el modelo `DecisionTreeRegressor`.

In [14]:
# Entrenando modelo
dtr_model = DecisionTreeRegressor(random_state=12345)
dtr_model.fit(features,target)

DecisionTreeRegressor(random_state=12345)

In [15]:
# Evaluando con validación cruzada
score = cross_val_score(dtr_model,features,target,cv=5)
print('Desempeño:',np.mean(score))

Desempeño: 0.9954002737197343


Observamos una gran diferencia entre el desempeño de los modelos de regresión tradicionales y la `regresión de árboles de decisión` pues esta tiene un desempeño considerablemente superior.

## Bosques aleatorios

Por último probaremos el modelo de `RandomForestRegressor`.

In [16]:
# Entrenando modelo
rfr_model = RandomForestRegressor(random_state=12345)
rfr_model.fit(features,target)

RandomForestRegressor(random_state=12345)

In [17]:
# Evaluando con validación cruzada
score = cross_val_score(rfr_model,features,target,cv=5)
print('Desempeño:',np.mean(score))

Desempeño: 0.9974211669336996


Para finalizar, observamos que el desempeño de este modelo es ligeramente superior al anteriormente mencionado, por tal motivo, haremos uso de este modelo para predecir la cantidad de beneficios que un cliente nuevo puede recibir basado en el entrenamiento ya realizado. Para probar el modelo haremos uso del cliente propuesto en la tarea 1.

In [18]:
# Predicción de beneficios de un cliente nuevo
prediction = rfr_model.predict([[1,43,41000,2]])
print(prediction)

[1.]


/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Compararemos este resultado con el resultado proporcionado por la función `nearest_neighbor_predict` para verificar que este resultado sea lo más cercano al correcto posible comparando los beneficios con los que cuentan los cliente con mayor similitud al nuevo cliente.

In [19]:
# Ejecutando "nearest_neighbor_predict"
idx_1, idx_2 = nearest_neighbor_predict(df,[1,43,41000,2],5)
df.iloc[idx_1.index,:], df.iloc[idx_2.index,:]

(      gender  age  salary  family_members  insurance_benefits
 5          1   43   41000               2                   1
 1147       1   42   40800               2                   0
 4074       1   43   39600               2                   1
 1019       1   42   39600               2                   0
 2962       1   41   41100               2                   0,
       gender  age  salary  family_members  insurance_benefits
 5          1   43   41000               2                   1
 1147       1   42   40800               2                   0
 4074       1   43   39600               2                   1
 2962       1   41   41100               2                   0
 1019       1   42   39600               2                   0)

De acuerdo con lo obtenido tanto de `nearest_neighbor_predict` como del modelo de regresión lineal, podemos observar congruencia en los resultados pues si bien tenemos un registro que es similar y que tiene un total de beneficios nulo, los 4 restantes tienen un beneficio lo que coincide con la predicción realizada anteriormente.

Por último y con la intención de implementar seguridad al algoritmo, implementaremos lo que se conoce como `enmascaramiento` u `ofuscación` de datos que es un proceso que nos permite encriptar la información para que se dificulte su lectura y entendimiento a cualquier persona que no este autorizada de conocerla pero sin perder la eficiencia anteriormente mostrada en los algoritmos lo que nos lleva a la tarea final.

# Tarea 4: Proteger los datos personales de los clientes sin romper el modelo de la tarea anterior.

Para proteger los datos personales que se manejan en `df` podemos hacer uso de diferentes técnicas y considerar muchos puntos dependiendo de la necesidad y nivel de protección. Una de las bibliotecamos más conocidas en python para poder llevar a cabo este proceso es `cryptography.fernet` que es una clase que nos permite hacer la encriptación y desencriptación haciendo uso de una llave o mejor conocido como `token`.

Este método nos permite poder cifrar y descifrar la información a través del uso del mismo `token`, por lo que haremos uso de ella y probaremos el modelo de regresión lineal.

In [20]:
# Aplicando le proceso de ofuscación
import hashlib
new_df = pd.DataFrame()
for column in df.columns:
    values = []
    for val in df[column]:
        hash_object = hashlib.sha256(str(val).encode('utf-8'))
        hash_value = int(hash_object.hexdigest(),16) % 1000000
        values.append(hash_value)
    new_df[column] = values

new_df.to_csv('datasets/insurance_us_ofuscated.csv',index=False)

In [21]:
# Implementando el modelo de regresión lineal
features = new_df.drop(columns=['insurance_benefits'],axis=1)
target = new_df['insurance_benefits']
new_rfr_model = RandomForestRegressor(random_state=12345)
new_rfr_model.fit(features,target)
score = cross_val_score(new_rfr_model,features,target,cv=5)
print('Desempeño:',np.mean(score))

Desempeño: 0.9350936889580428


De acuerdo con el desempeño que el algoritmo de regresión lineal tiene una vez si aplico da `ofuscación` a los datos, vemos que sigue teniendo un desempeño elevado que si bien este se vió disminuido aproximadamente un `6%` esto es de entenderse debido a los datos de la `ofuscación` propician a un rendimiento un poco más bajo por la transformación que sufren los datos a lo largo de este proceso.

# Conclusión

De manera general se lograrón cumplir con las 4 tareas designadas por la compañia `Sure Tomorrow` en las que se pudo demostrar que podemos encontrar clientes con características similares a uno en específico ayudando a la compañia con campañas de marketing. Por otro lado pudimos verficiar que si hay clientes que tienen características similares a otros que ya hay recibido beneficios por parte del seguro, es probable que de igual forma reciban un beneficio aunque estos sean clientes nuevos.

Al implementar un modelo de `regresión lineal` podemos de igual forma predecir la cantidad de beneficios que un cliente puede llegar a tener con precisión adecuada incluso aplicando `ofuscación` a los registros para que estos se encuentren protegidos.